In [4]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import numpy as np
import torch

t_c = torch.tensor([0.5,14.0,15.0,28.0,11.0,8.0,3.0,-4.0,6.0,13.0,21.0])
t_u = torch.tensor([35.7,55.9,58.2,81.9,56.3,48.9,33.9,21.8,48.4,60.4,68.4])
print(t_u.shape)

t_c = t_c.unsqueeze(1)
t_u = t_u.unsqueeze(1)

n_samples = t_u.shape[0]
n_val = int(0.2*n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices=shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

print(train_indices,val_indices)

t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1*t_u_train
t_un_val = 0.1*t_u_val

torch.Size([11])
tensor([ 9,  6,  5,  7,  1,  4,  8,  0, 10]) tensor([3, 2])


In [22]:
import torch.nn as nn

linear_model = nn.Linear(1,1)
linear_model(val_t_un)

tensor([[4.8320],
        [3.1590]], grad_fn=<AddmmBackward>)

In [23]:
linear_model.weight

Parameter containing:
tensor([[0.8122]], requires_grad=True)

In [24]:
linear_model.bias

Parameter containing:
tensor([0.2596], requires_grad=True)

In [25]:
x = torch.ones(10,1)
linear_model(x)

tensor([[1.0718],
        [1.0718],
        [1.0718],
        [1.0718],
        [1.0718],
        [1.0718],
        [1.0718],
        [1.0718],
        [1.0718],
        [1.0718]], grad_fn=<AddmmBackward>)

In [26]:
import torch.optim as optim

linear_model = nn.Linear(1,1)
optimizer = optim.SGD(
    linear_model.parameters(),
    lr=1e-2
)

In [27]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[-0.6227]], requires_grad=True), Parameter containing:
 tensor([-0.9424], requires_grad=True)]

In [28]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train,
                  t_u_val,t_c_train,t_c_val):
  for epoch in range(1,n_epochs+1):
    t_p_train = model(t_u_train)
    loss_train = loss_fn(t_p_train, t_c_train)

    t_p_val = model(t_u_val)
    loss_val = loss_fn(t_p_val,t_c_val)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch==1 or epoch%1000==0:
      print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
            f" Validation loss {loss_val.item():.4f}")

In [29]:
def loss_fn(t_p,t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()

In [30]:
linear_model = nn.Linear(1,1)
optimizer = optim.SGD(linear_model.parameters(),lr=1e-2)

training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    model=linear_model,
    loss_fn=loss_fn,
    t_u_train=t_un_train,
    t_u_val=t_un_val,
    t_c_train=t_c_train,
    t_c_val = t_c_val
)

Epoch 1, Training loss 83.5016, Validation loss 384.2205
Epoch 1000, Training loss 3.8950, Validation loss 13.4908
Epoch 2000, Training loss 3.0178, Validation loss 6.6178
Epoch 3000, Training loss 2.9747, Validation loss 5.4433


In [31]:
print()
print(linear_model.weight)
print(linear_model.bias)


Parameter containing:
tensor([[5.0071]], requires_grad=True)
Parameter containing:
tensor([-15.8379], requires_grad=True)
